# 🐍 Setup Focoos

In [ ]:
%pip install 'focoos @ git+https://github.com/FocoosAI/focoos.git'

# If you want to run it locally using CPU you can install the package with the following command:
# %pip install 'focoos[cpu] @ git+https://github.com/FocoosAI/focoos.git'

# If you want to run it locally using GPU you can install the package with the following command:
# %pip install 'focoos[cuda] @ git+https://github.com/FocoosAI/focoos.git'

In [ ]:
import os

if not os.path.exists("image.jpg"):
    print("Downloading image.jpg")
    !curl https://www.ondacinema.it/images/serial/xl/howimetyourmother-fotoxl.jpg -o image.jpg
image_path = "image.jpg"

# 🎨 Personalize your model

This section covers the steps to create a model and train it in the cloud using the focoos library. The following example demonstrates how to interact with the Focoos API to manage models, datasets, and training jobs.

In this guide, we will perform the following steps:

0. 🐍 Connect with Focoos
1. 📦 Load or select a dataset
2. 🎯 Create a model
3. 🏃‍♂️ Train the model
4. 📊 Visualize training metrics
5. 🧪 Test your model


## 🐍 Connect with Focoos

In [ ]:
from pprint import pprint

from PIL import Image

from focoos import Focoos

focoos = Focoos(api_key="<YOUR-API-KEY>")

## 📦 Load or select a dataset

Currently, we are not supporting dataset creation from the SDK (it's coming really soon) and you can only use a dataset already available on the platform. To upload your own dataset, you can write us a mail to info@focoos.ai and we will load your dataset on the platform on your private workspace (your data will not be shared with anyone and not used for any other purpose than training your model).


In [ ]:
datasets = focoos.list_shared_datasets()

for dataset in datasets:
    print(f"Name: {dataset.name}")
    print(f"Reference: {dataset.ref}")
    print(f"Task: {dataset.task}")
    print(f"Description: {dataset.description}")
    print("-" * 50)

In [ ]:
# select the dataset you want to use for training
dataset_ref = "7b7c0ed8cf804f1d"

## 🎯 Create a model

The first step to personalize your model is to create a model. You can create a model by calling the new_model method on the Focoos object. You can choose the model you want to personalize from the list of Focoos Models available on the platform. Make sure to select the correct model for your task.

In [ ]:
model = focoos.new_model(
    name="my-model",
    description="my-model-description",
    focoos_model="fai-rtdetr-m-obj365",
)

This function returns a new RemoteModel object that you can use to train the model and to perform remote inference.

## 🏃‍♂️ Train the model

The next step is to train the model. You can train the model by calling the train method on the RemoteModel object. You can choose the dataset you want to use for training and the instance type you want to use for training.

In [ ]:
from focoos import Hyperparameters

res = model.train(
    dataset_ref=dataset_ref,
    hyperparameters=Hyperparameters(
        learning_rate=0.0001,  # custom learning rate
        batch_size=16,  # custom batch size
        max_iters=1500,  # custom max iterations
    ),
)

We provide you with a notebook monitor to track the training progress. You can use it to monitor the training progress and to get the training logs.

In [ ]:
model.notebook_monitor_train(interval=30, plot_metrics=True)

Or if you prefer the hard way, you can get the training logs by calling the train_logs method on the RemoteModel object.

In [ ]:
logs = model.train_logs()
pprint(logs)

If for some reason you need to stop the training, you can do so by calling the stop_train method on the RemoteModel object.

In [ ]:
model.stop_training()

## 📊 Visualize training metrics

In [ ]:
from focoos.utils.metrics import MetricsVisualizer

metrics = model.metrics()
visualizer = MetricsVisualizer(metrics)
visualizer.log_metrics()

In [ ]:
visualizer.notebook_plot_training_metrics()

## 🧪 Test your model


### Remote Inference

In [ ]:
result, preview = model.infer(image_path, threshold=0.6, annotate=True)

Image.fromarray(preview)

### Local Inference
Remember, to perform local inference, you need to install the package with one of the extra modules (`[cpu]`, `[torch]`, `[cuda]`, `[tensorrt]`). See the [installation](./setup.md) page or the `inference.ipynb` notebook for more details.

In [ ]:
%pip install 'focoos[cpu] @ git+https://github.com/FocoosAI/focoos.git'
# Rerun the kernel to reload the modules with the new dependencies

In [ ]:
model = focoos.get_local_model(model.model_ref)  # get the local model

result, _ = model.infer(image_path, threshold=0.5, annotate=False)

for det in result.detections:
    print(f"Found {det.label} with confidence {det.conf:.2f}")
    print(f"Bounding box: {det.bbox}")
    if det.mask:
        print("Instance segmentation mask included")